# Construct enrollment-by-bid datasets

## 2021

In [ ]:
# Import data

df_Section_A = pd.read_csv('PBP Benefits 2021/pbp_Section_A.txt', sep='\t', encoding = "ISO-8859-1", dtype='str')
df_area = pd.read_csv('PBP Benefits 2021/PlanArea.txt', sep='\t', encoding = "ISO-8859-1",  dtype='str')
df_enrollment = pd.read_csv('CPSC_Enrollment_Info_2021_03.csv', encoding = "ISO-8859-1",  dtype='str')

In [ ]:
# Add geographical footprint

df_area = df_area.drop_duplicates(subset=['bid_id', 'county_code'])
df = df_Section_A.merge(right=df_area[['bid_id', 'county_code']], on='bid_id', how='left')

In [ ]:
# Add enrollment

df_enrollment =df_enrollment[df_enrollment.Enrollment!='*']
df_hmo = df[df.pbp_a_hnumber.str.slice(0,1)!='R']
df_hmo = df_hmo.merge(right = df_enrollment, left_on = ['pbp_a_hnumber', 'pbp_a_plan_identifier', 'county_code'], 
             right_on = ['Contract Number', 'Plan ID', 'SSA State County Code'], how = 'left')
df_ppo = df[df.pbp_a_hnumber.str.slice(0,1)=='R']
df_ppo = df_ppo.merge(right = df_enrollment, left_on = ['pbp_a_hnumber', 'pbp_a_plan_identifier'], 
             right_on = ['Contract Number', 'Plan ID'], how = 'left')
df = pd.concat([df_hmo,df_ppo])
df=df[~df.Enrollment.isna()]
df.Enrollment=pd.to_numeric(df.Enrollment)

In [ ]:
# Enrollment by bid

df_enrollment_2021 = df.groupby('bid_id').Enrollment.sum()

## 2020

In [318]:
# Import data

df_Section_A = pd.read_csv('PBP Benefits 2020/pbp_Section_A.txt', sep='\t', encoding = "ISO-8859-1", dtype='str')
df_area = pd.read_csv('PBP Benefits 2020/PlanArea.txt', sep='\t', encoding = "ISO-8859-1",  dtype='str')
df_enrollment = pd.read_csv('CPSC_Enrollment_Info_2020_03.csv', encoding = "ISO-8859-1",  dtype='str')

In [319]:
# Add geographical footprint

df_area = df_area.drop_duplicates(subset=['bid_id', 'county_code'])
df = df_Section_A.merge(right=df_area[['bid_id', 'county_code']], on='bid_id', how='left')

In [320]:
# Add enrollment

df_enrollment =df_enrollment[df_enrollment.Enrollment!='*']
df_hmo = df[df.pbp_a_hnumber.str.slice(0,1)!='R']
df_hmo = df_hmo.merge(right = df_enrollment, left_on = ['pbp_a_hnumber', 'pbp_a_plan_identifier', 'county_code'], 
             right_on = ['Contract Number', 'Plan ID', 'SSA State County Code'], how = 'left')
df_ppo = df[df.pbp_a_hnumber.str.slice(0,1)=='R']
df_ppo = df_ppo.merge(right = df_enrollment, left_on = ['pbp_a_hnumber', 'pbp_a_plan_identifier'], 
             right_on = ['Contract Number', 'Plan ID'], how = 'left')
df = pd.concat([df_hmo,df_ppo])
df=df[~df.Enrollment.isna()]
df.Enrollment=pd.to_numeric(df.Enrollment)

In [321]:
# Enrollment by bid

df_enrollment_2020 = df.groupby('bid_id').Enrollment.sum()

In [322]:
df_enrollment_2020.sum()

24521203

# Primarily health related

## 2021

### All new benefits

In [ ]:
#Import data

df_Section_A = pd.read_csv('PBP Benefits 2021/pbp_Section_A.txt', sep='\t', encoding = "ISO-8859-1", dtype='str')
df_b14 = pd.read_csv('PBP Benefits 2021/pbp_b14_preventive.txt', sep='\t', encoding = "ISO-8859-1",  dtype='str')
df_b10 = pd.read_csv('PBP Benefits 2021/pbp_b10_amb_trans.txt', sep='\t', encoding = "ISO-8859-1",  dtype='str')
df_b13 = pd.read_csv('PBP Benefits 2021/pbp_b13_other_services.txt', sep='\t', encoding = "ISO-8859-1",  dtype='str')

In [ ]:
# Identify what plans have each benefit

df_day_care = df_b14[df_b14.pbp_b14c_bendesc_ehc.str.contains("14c19;", na=False)]
df_palliative = df_b14[df_b14.pbp_b14c_bendesc_ehc.str.contains("14c20;", na=False)]
df_home_support = df_b14[df_b14.pbp_b14c_bendesc_ehc.str.contains("14c21;", na=False)]
df_caregiver_support = df_b14[df_b14.pbp_b14c_bendesc_ehc.str.contains("14c22;", na=False)]
df_pain_mgmt  = df_b14[df_b14.pbp_b14c_bendesc_ehc.str.contains("14c18;", na=False)]
df_home_mod = df_b14[df_b14.pbp_b14c_bendesc_ehc.str.contains("14c8;", na=False)]
df_transport = df_b10[df_b10.pbp_b10b_bendesc_yn=='1']
df_otc = df_b13[df_b13.pbp_b13b_bendesc_otc=='1']
l= [df_day_care, df_palliative, df_home_support, df_caregiver_support, df_pain_mgmt, df_home_mod, df_transport, df_otc]

In [ ]:
# Create bid level benefits dataset

df=df_Section_A
df=df[df.pbp_a_hnumber.str.slice(0,1) !='S'] # get rid of PDPs
for frame in l:
    frame.name =[x for x in globals() if globals()[x] is frame][0]
    name = 'stub_' + frame.name[3:]
    df=df.merge(right=frame.bid_id, on='bid_id', how='left', indicator=name)
    df[name] = df[name].replace(['both', 'left_only'],['1', '0'])

In [ ]:
# Add enrollment

df=df.merge(df_enrollment_2021, on='bid_id', how='left')

In [ ]:
# Wide to long

df=pd.wide_to_long(df, stubnames='stub_', i='bid_id', j='benefit', suffix= '\D+').reset_index()

In [ ]:
# Output plans (plan contract combo)

df['plan']=df.bid_id.str.slice(0,9)
df[df.stub_=="1"].groupby(['benefit']).plan.nunique().to_clipboard()

In [ ]:
# Output enrollment

df[df.stub_=="1"].groupby(['benefit']).Enrollment.sum().to_clipboard()

### Deep-dive on in-home supports

In [ ]:
df = pd.read_csv('PBP Benefits 2021/pbp_Section_A.txt', sep='\t', encoding = "ISO-8859-1", dtype='str')
df_b14 = pd.read_csv('PBP Benefits 2021/pbp_b14_preventive.txt', sep='\t', encoding = "ISO-8859-1",  dtype='str')
df_area = pd.read_csv('PBP Benefits 2021/PlanArea.txt', sep='\t', encoding = "ISO-8859-1",  dtype='str')
df_enrollment = pd.read_csv('CPSC_Enrollment_Info_2021_03.csv', encoding = "ISO-8859-1",  dtype='str')
df_details = pd.read_csv('CPSC_Contract_Info_2021_03.csv', encoding = "ISO-8859-1",  dtype='str')

In [ ]:
df_home_support = df_b14[df_b14.pbp_b14c_bendesc_ehc.str.contains("14c21;", na=False)]
df=df.merge(right=df_home_support.bid_id, on='bid_id', how='left', indicator='home_support')
df.home_support= df.home_support.replace(['both', 'left_only'],[1, 0])

In [ ]:
df_area = df_area.drop_duplicates(subset=['bid_id', 'county_code'])
df = df.merge(right=df_area[['bid_id', 'county_code']], on='bid_id', how='left')

In [ ]:
df_enrollment =df_enrollment[df_enrollment.Enrollment!='*']
df_hmo = df[df.pbp_a_hnumber.str.slice(0,1)!='R']
df_hmo = df_hmo.merge(right = df_enrollment, left_on = ['pbp_a_hnumber', 'pbp_a_plan_identifier', 'county_code'], 
             right_on = ['Contract Number', 'Plan ID', 'SSA State County Code'], how = 'left')
df_hmo = df_hmo [~df_hmo.Enrollment.isna()]
df_ppo = df[df.pbp_a_hnumber.str.slice(0,1)=='R']
df_ppo = df_ppo.merge(right = df_enrollment, left_on = ['pbp_a_hnumber', 'pbp_a_plan_identifier'], 
             right_on = ['Contract Number', 'Plan ID'], how = 'left')
df_ppo = df_ppo[~df_ppo.Enrollment.isna()]
df = pd.concat([df_hmo,df_ppo])
df.Enrollment = pd.to_numeric(df.Enrollment)

In [ ]:
pd.options.display.max_rows = 4000
df=df.merge(right=df_details, left_on=['pbp_a_hnumber', 'pbp_a_plan_identifier'], right_on=['Contract ID', 'Plan ID'], how='left')
df.groupby(['Parent Organization']).Enrollment.sum()

In [ ]:
df[df.home_support==1].groupby(['Parent Organization']).Enrollment.sum()

In [ ]:
df=df[df['FIPS State County Code']=='12086']
df.groupby(['Parent Organization']).Enrollment.sum()

In [ ]:
df[df.home_support==1].groupby(['Parent Organization']).Enrollment.sum()

## 2020

In [ ]:
#Import data

df_Section_A = pd.read_csv('PBP Benefits 2020/pbp_Section_A.txt', sep='\t', encoding = "ISO-8859-1", dtype='str')
df_b14 = pd.read_csv('PBP Benefits 2020/pbp_b14_preventive.txt', sep='\t', encoding = "ISO-8859-1",  dtype='str')
df_b10 = pd.read_csv('PBP Benefits 2020/pbp_b10_amb_trans.txt', sep='\t', encoding = "ISO-8859-1",  dtype='str')
df_b13 = pd.read_csv('PBP Benefits 2020/pbp_b13_other_services.txt', sep='\t', encoding = "ISO-8859-1",  dtype='str')

In [ ]:
# Identify what plans have each benefit

df_day_care = df_b14[df_b14.pbp_b14c_bendesc_ehc.str.contains("14c19;", na=False)]
df_palliative = df_b14[df_b14.pbp_b14c_bendesc_ehc.str.contains("14c20;", na=False)]
df_home_support = df_b14[df_b14.pbp_b14c_bendesc_ehc.str.contains("14c21;", na=False)]
df_caregiver_support = df_b14[df_b14.pbp_b14c_bendesc_ehc.str.contains("14c22;", na=False)]
df_pain_mgmt  = df_b14[df_b14.pbp_b14c_bendesc_ehc.str.contains("14c18;", na=False)]
df_home_mod = df_b14[df_b14.pbp_b14c_bendesc_ehc.str.contains("14c8;", na=False)]
df_transport = df_b10[df_b10.pbp_b10b_bendesc_yn=='1']
df_otc = df_b13[df_b13.pbp_b13b_bendesc_otc=='1']
l= [df_day_care, df_palliative, df_home_support, df_caregiver_support, df_pain_mgmt, df_home_mod, df_transport, df_otc]

In [ ]:
# Create bid level benefits dataset

df=df_Section_A
df=df[df.pbp_a_hnumber.str.slice(0,1) !='S'] # get rid of PDPs
for frame in l:
    frame.name =[x for x in globals() if globals()[x] is frame][0]
    name = 'stub_' + frame.name[3:]
    df=df.merge(right=frame.bid_id, on='bid_id', how='left', indicator=name)
    df[name] = df[name].replace(['both', 'left_only'],['1', '0'])

In [ ]:
# Add enrollment

df=df.merge(df_enrollment_2020, on='bid_id', how='left')

In [ ]:
# Wide to long

df=pd.wide_to_long(df, stubnames='stub_', i='bid_id', j='benefit', suffix= '\D+').reset_index()

In [ ]:
# Output enrollment

df[df.stub_=="1"].groupby(['benefit']).Enrollment.sum().to_clipboard()

# SSBCI

## 2021

### All benefits 

In [ ]:
#Import data

df_b13_b19 = pd.read_csv('PBP Benefits 2021/pbp_b13i_b19b_services_vbid_ssbci.txt', sep='\t', encoding = "ISO-8859-1",  dtype='str')

In [ ]:
# Construct benefits dataset

df=pd.DataFrame({'key':[y for x in [[l]*10 for l in df_b13_b19.pbp_b13i_bendesc] for y in x]})
df['bid_id']=[y for x in [[l]*10 for l in df_b13_b19.bid_id] for y in x]
df['benefit']=['General Supports for Living','Food and Produce','Meals (beyond limited basis)',
'Pest Control','Transportation for Non-Medical Needs','Indoor Air Quality Equipment and Services',
'Social Needs Benefit','Complementary Therapies','Services Supporting Self-Direction',
'Structural Home Modifications']*df.bid_id.nunique()
df['pos']=list(range(0,10))*df.bid_id.nunique()
df['offered']=df.apply(lambda x: x['key'][x['pos']], axis=1)
df=df[df.offered=="1"]

In [ ]:
# Output plans (plan contract combo)

df['plan']=df.bid_id.str.slice(0,9)
df.groupby(['benefit']).plan.nunique().to_clipboard()

In [ ]:
# Get enrollment by benefit

df=df.merge(df_enrollment_2021, on='bid_id', how='inner')
df.groupby('benefit').Enrollment.sum()#.to_clipboard()

### Deep dive on meals

In [ ]:
# Narrow to bids with meal benefit and add enrollment

df_meals=df_b13_b19[df_b13_b19.pbp_b13i_ml_bendesc_service=="1"]
df_meals=df_meals.merge(df_enrollment_2021, on='bid_id', how='inner')

In [ ]:
# Average days

np.dot(pd.to_numeric(df_meals.pbp_b13i_ml_days),df_meals.Enrollment)/df_meals.Enrollment.sum()

In [ ]:
# Average meals

np.dot(pd.to_numeric(df_meals.pbp_b13i_ml_max_meals),df_meals.Enrollment)/df_meals.Enrollment.sum()

In [ ]:
# Percent authorization

df_meals[df_meals.pbp_b13i_ml_auth_yn=='1'].Enrollment.sum()/df_meals.Enrollment.sum()

In [ ]:
# Percent referral

df_meals[df_meals.pbp_b13i_ml_refer_yn=='1'].Enrollment.sum()/df_meals.Enrollment.sum()

### Deep dive on transport

In [ ]:
# Narrow to bids with transport benefit and add enrollment

df_t=df_b13_b19[df_b13_b19.pbp_b13i_t_bendesc_yn=="1"]
df_t=df_t.merge(df_enrollment_2021, on='bid_id', how='inner')

#### Plan-approve only

In [ ]:
# Percent plan-approved only

df_t1=df_t[df_t.pbp_b13i_t_bendesc_trn=='1']
df_t1.Enrollment.sum()/df_t.Enrollment.sum()

In [ ]:
# Of that, percent limited

df_t2=df_t1[df_t1.pbp_b13i_t_bendesc_lim_pal!='1']
df_t2.Enrollment.sum()/df_t1.Enrollment.sum()

In [ ]:
# Of limited (but not zero), average trips

df_t3=df_t2[df_t2.pbp_b13i_t_bendesc_amt_pal!='0']
pd.to_numeric(df_t3.pbp_b13i_t_bendesc_amt_pal).dot(df_t3.Enrollment)/df_t3.Enrollment.sum()

In [ ]:
# Create transport mode dataset 

df_mode=pd.DataFrame({'key':[y for x in [[l]*6 for l in df_t1.pbp_b13i_t_bendesc_mt_pal] for y in x]})
df_mode['bid_id']=[y for x in [[l]*6 for l in df_t1.bid_id] for y in x]
df_mode['mode']=['Other', 'Taxi', 'Rideshare', 'Bus/subway', 'Van', 'Med transport']*len(df_t1)
df_mode['pos']=list(range(0,6))*len(df_t1)
df_mode['offered']= (df_mode.apply(lambda x: x['key'][x['pos']], axis=1) =="1")*1
df_mode= df_mode[df_mode.offered==1]

In [ ]:
# Add enrollment and export

df_mode=df_mode.merge(df_enrollment_2021, on='bid_id', how='inner')
df_mode.groupby('mode').Enrollment.apply(lambda x: x.sum()/df_t1.Enrollment.sum()).to_clipboard()

#### Any location

In [ ]:
# Percent any location

df_t1=df_t[df_t.pbp_b13i_t_bendesc_trn=='2']
df_t1.Enrollment.sum()/df_t.Enrollment.sum()

In [ ]:
# Of that, percent limited

df_t2=df_t1[df_t1.pbp_b13i_t_bendesc_lim_al!='1']
df_t2.Enrollment.sum()/df_t1.Enrollment.sum()

In [ ]:
# Of limited (but not zero), average trips

df_t3=df_t2[df_t2.pbp_b13i_t_bendesc_amt_al!='0']
pd.to_numeric(df_t3.pbp_b13i_t_bendesc_amt_al).dot(df_t3.Enrollment)/df_t3.Enrollment.sum()

In [ ]:
# Create transport mode dataset 

df_mode=pd.DataFrame({'key':[y for x in [[l]*6 for l in df_t1.pbp_b13i_t_bendesc_mt_al] for y in x]})
df_mode['bid_id']=[y for x in [[l]*6 for l in df_t1.bid_id] for y in x]
df_mode['mode']=['Other', 'Taxi', 'Rideshare', 'Bus/subway', 'Van', 'Med transport']*len(df_t1)
df_mode['pos']=list(range(0,6))*len(df_t1)
df_mode['offered']= (df_mode.apply(lambda x: x['key'][x['pos']], axis=1) =="1")*1
df_mode= df_mode[df_mode.offered==1]

In [ ]:
# Add enrollment and export

df_mode=df_mode.merge(df_enrollment_2021, on='bid_id', how='inner')
df_mode.groupby('mode').Enrollment.apply(lambda x: x.sum()/df_t1.Enrollment.sum()).to_clipboard()

## 2020

### All benefits 

In [ ]:
#Import data

df_b13_b19 = pd.read_csv('PBP Benefits 2020/pbp_b13i_b19b_services_vbid_ssbci.txt', sep='\t', encoding = "ISO-8859-1",  dtype='str')

In [ ]:
# Construct benefits dataset

df=pd.DataFrame({'key':[y for x in [[l]*10 for l in df_b13_b19.pbp_b13i_bendesc] for y in x]})
df['bid_id']=[y for x in [[l]*10 for l in df_b13_b19.bid_id] for y in x]
df['benefit']=['General Supports for Living','Food and Produce','Meals (beyond limited basis)',
'Pest Control','Transportation for Non-Medical Needs','Indoor Air Quality Equipment and Services',
'Social Needs Benefit','Complementary Therapies','Services Supporting Self-Direction',
'Structural Home Modifications']*df.bid_id.nunique()
df['pos']=list(range(0,10))*df.bid_id.nunique()
df['offered']=df.apply(lambda x: x['key'][x['pos']], axis=1)
df=df[df.offered=="1"]

In [ ]:
# Count plans

df.groupby(['benefit']).bid_id.count() #.to_clipboard()

In [ ]:
# Get enrollment by benefit

df=df.merge(df_enrollment_2020, on='bid_id', how='inner')
df.groupby('benefit').Enrollment.sum()#.to_clipboard()

# Telehealth

## 2021

In [313]:
# Import data and create dict to translate key

df_b7 = pd.read_csv('PBP Benefits 2021/pbp_b7_health_prof.txt', sep='\t', encoding = "ISO-8859-1",  dtype='str')
df_key= pd.read_excel('PBP Benefits 2021/PBP_Benefits_2021_dictionary.xlsx')
df_key=pd.DataFrame(df_key[df_key.NAME=='pbp_b7j_mc_bendesc_cats'].CODE_VALUES.str.split(':').tolist(), columns=['code', 'benefit'])

In [314]:
# Construct benefit-level dataset

df_b7=df_b7[['pbp_b7j_mc_bendesc_cats', 'bid_id']].dropna()
df_b7['count'] = df_b7.pbp_b7j_mc_bendesc_cats.str.count(';')
df_b7.pbp_b7j_mc_bendesc_cats = df_b7.pbp_b7j_mc_bendesc_cats.str.slice(0,-1)
df_b7.pbp_b7j_mc_bendesc_cats = df_b7.pbp_b7j_mc_bendesc_cats.str.split(';')
list_of_rows=[df_b7.loc[i] for i in df_b7.index]
df=pd.DataFrame({'code':[x for y in df_b7.pbp_b7j_mc_bendesc_cats for x in y],
                 'bid_id':[x for y in [[row['bid_id']]*row['count'] for row in list_of_rows] for x in y]})
df=df.merge(df_key, on='code', how='left')

In [315]:
# Output plans (plan contract combo)

df['plan']=df.bid_id.str.slice(0,9)
df.groupby('benefit').plan.nunique().to_clipboard()

In [316]:
# Add enrollment and export

df= df.join(df_enrollment_2021, on='bid_id', how='left')
df.groupby('benefit').Enrollment.sum().to_clipboard()

### Deep dive on Individual sessions for mental health specialty services

In [ ]:
# Import data

df_area = pd.read_csv('PBP Benefits 2021/PlanArea.txt', sep='\t', encoding = "ISO-8859-1",  dtype='str')
df_enrollment = pd.read_csv('CPSC_Enrollment_Info_2021_03.csv', encoding = "ISO-8859-1",  dtype='str')
df_enrollment.Enrollment = pd.to_numeric(df_enrollment.Enrollment, errors='coerce')
df_enrollment=df_enrollment.dropna()

In [ ]:
# Get bids offering benefit

df=df_b7[['pbp_a_hnumber','pbp_a_plan_identifier','bid_id', 'pbp_b7j_mc_bendesc_cats']]
df=df.dropna()
df=df[df.pbp_b7j_mc_bendesc_cats.str.contains("7e1;")]

In [ ]:
# Add enrollment by county

df_hmo=df[df.bid_id.str.slice(0,1)!="R"]
df_hmo = df_hmo.merge(right=df_area[['bid_id', 'county_code']], on='bid_id', how='left')
df_hmo = df_hmo.merge(right = df_enrollment, left_on = ['pbp_a_hnumber', 'pbp_a_plan_identifier', 'county_code'], 
             right_on = ['Contract Number', 'Plan ID', 'SSA State County Code'], how = 'inner')

df_ppo=df[df.bid_id.str.slice(0,1)=="R"]
df_ppo = df_ppo.merge(right = df_enrollment, left_on = ['pbp_a_hnumber', 'pbp_a_plan_identifier'], 
             right_on = ['Contract Number', 'Plan ID'], how = 'inner')
df=pd.concat([df_hmo,df_ppo])

In [ ]:
# Export

df.groupby('FIPS State County Code').Enrollment.sum().to_clipboard()

## 2020

In [309]:
# Import data and create dict to translate key

df_b7 = pd.read_csv('PBP Benefits 2020/pbp_b7_health_prof.txt', sep='\t', encoding = "ISO-8859-1",  dtype='str')
df_key= pd.read_excel('PBP Benefits 2020/PBP_Benefits_2020_dictionary.xlsx')
df_key=pd.DataFrame(df_key[df_key.NAME=='pbp_b7j_mc_bendesc_cats'].CODE_VALUES.str.split(':').tolist(), columns=['code', 'benefit'])

In [310]:
# Construct benefit-level dataset

df_b7=df_b7[['pbp_b7j_mc_bendesc_cats', 'bid_id']].dropna()
df_b7['count'] = df_b7.pbp_b7j_mc_bendesc_cats.str.count(';')
df_b7.pbp_b7j_mc_bendesc_cats = df_b7.pbp_b7j_mc_bendesc_cats.str.slice(0,-1)
df_b7.pbp_b7j_mc_bendesc_cats = df_b7.pbp_b7j_mc_bendesc_cats.str.split(';')
list_of_rows=[df_b7.loc[i] for i in df_b7.index]
df=pd.DataFrame({'code':[x for y in df_b7.pbp_b7j_mc_bendesc_cats for x in y],
                 'bid_id':[x for y in [[row['bid_id']]*row['count'] for row in list_of_rows] for x in y]})
df=df.merge(df_key, on='code', how='left')

In [311]:
# Output plans (plan contract combo)

df['plan']=df.bid_id.str.slice(0,9)
df.groupby('benefit').plan.nunique().to_clipboard()

In [312]:
# Add enrollment and export

df= df.join(df_enrollment_2020, on='bid_id', how='left')
df.groupby('benefit').Enrollment.sum().to_clipboard()

### Deep dive on Individual sessions for mental health specialty services

In [ ]:
# Import data

df_area = pd.read_csv('PBP Benefits 2020/PlanArea.txt', sep='\t', encoding = "ISO-8859-1",  dtype='str')
df_enrollment = pd.read_csv('CPSC_Enrollment_Info_2020_03.csv', encoding = "ISO-8859-1",  dtype='str')
df_enrollment.Enrollment = pd.to_numeric(df_enrollment.Enrollment, errors='coerce')
df_enrollment=df_enrollment.dropna()

In [ ]:
# Get bids offering benefit

df=df_b7[['pbp_a_hnumber','pbp_a_plan_identifier','bid_id', 'pbp_b7j_mc_bendesc_cats']]
df=df.dropna()
df=df[df.pbp_b7j_mc_bendesc_cats.str.contains("7e1;")]

In [ ]:
# Add enrollment by county

df_hmo=df[df.bid_id.str.slice(0,1)!="R"]
df_hmo = df.merge(right=df_area[['bid_id', 'county_code']], on='bid_id', how='left')
df_hmo = df_hmo.merge(right = df_enrollment, left_on = ['pbp_a_hnumber', 'pbp_a_plan_identifier', 'county_code'], 
             right_on = ['Contract Number', 'Plan ID', 'SSA State County Code'], how = 'inner')

df_ppo=df[df.bid_id.str.slice(0,1)=="R"]
df_ppo = df_ppo.merge(right = df_enrollment, left_on = ['pbp_a_hnumber', 'pbp_a_plan_identifier'], 
             right_on = ['Contract Number', 'Plan ID'], how = 'inner')
df=pd.concat([df_hmo,df_ppo])

In [ ]:
#Export

df.groupby('FIPS State County Code').Enrollment.sum().to_clipboard()

# Traditional benefits 

In [175]:
#Import data

df_b14 = pd.read_csv('PBP Benefits 2021/pbp_b14_preventive.txt', sep='\t', encoding = "ISO-8859-1",  dtype='str')
df_b16 = pd.read_csv('PBP Benefits 2021/pbp_b16_dental.txt', sep='\t', encoding = "ISO-8859-1",  dtype='str')
df_b17 = pd.read_csv('PBP Benefits 2021/pbp_b17_eye_exams_wear.txt', sep='\t', encoding = "ISO-8859-1",  dtype='str')
df_b18 = pd.read_csv('PBP Benefits 2021/pbp_b18_hearing_exams_aids.txt', sep='\t', encoding = "ISO-8859-1",  dtype='str')

In [165]:
# Isolate bids with each benefit and construct benefits dataset

fitness=df_b14[df_b14.pbp_b14c_bendesc_ehc.str.contains('14c4;',na=False)].bid_id
dental=df_b16[(df_b16.pbp_b16a_bendesc_yn=='1')| (df_b16.pbp_b16b_bendesc_yn=='1')].bid_id
vision=df_b17[(df_b17.pbp_b17a_bendesc_yn=='1')| (df_b17.pbp_b17b_bendesc_yn=='1')].bid_id
hearing=df_b18[(df_b18.pbp_b18a_bendesc_yn=='1')|(df_b18.pbp_b18b_bendesc_yn=='1')].bid_id

df=pd.DataFrame(columns=['bid_id', 'benefit'])
for bene in ['fitness', 'dental', 'vision', 'hearing']:
    df=pd.concat([df,pd.DataFrame({'bid_id' : globals()[bene], 'benefit': [bene]*len(globals()[bene])})])

In [166]:
# Add enrollment

df=df.merge(df_enrollment_2021, on='bid_id',how='left')

In [169]:
df.groupby('benefit').Enrollment.sum()/ df_enrollment_2021.sum()

benefit
dental     0.759824
fitness    0.742343
hearing    0.779285
vision     0.803471
Name: Enrollment, dtype: float64

In [172]:
# Output plans (plan contract combo)

df['plan']=df.bid_id.str.slice(0,9)
df.groupby(['benefit']).plan.nunique()/len(df_enrollment_2021)

benefit
dental     0.805765
fitness    0.815438
hearing    0.828787
vision     0.871735
Name: plan, dtype: float64

In [ ]:
[df]*3